In [ ]:
#단순 최다 빈도 명사
# word_cnt = Counter(nouns).most_common()
# total_df = pd.DataFrame(word_cnt, columns = ['단어', '빈도수'])
# total_df.head(10)

In [1]:
import json
import csv
import pandas as pd
from konlpy.tag import Komoran
from konlpy.corpus import kolaw
from konlpy.tag import Okt
import nltk
from nltk import Text
from collections import Counter
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from collections import defaultdict
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('natepann_Morphs.csv')
data.values
values = "".join(str(i) for i in data.values)

#okt = Okt()
komoran = Komoran() 

#total_morphs = okt.morphs(values)
total_morphs = komoran.morphs(values)

print("전체 형태소 개수: ")
print(len(total_morphs))

#nouns = okt.nouns(values)
nouns = komoran.nouns(values)
print("전체 명사 개수: ")
print(len(nouns))

nouns = [noun for noun in nouns if len(noun) >= 2] #3글자 이상의 키워드를 뽑는 것도 고려해 볼 것

bool_words = ['아니', '아서', '아도', '진짜', '생각', '라고', '지만', '정도', '만나', '제가', '지금', 
             '하나', '보이', '얘기', '마음', '우리', '사람', '가지', '생기', '잘못', '때문', '이랑', '문제', '네이트판']

#unique_Nouns = set(nouns)
for word in nouns:
    if word in bool_words:
        while word in nouns: 
            nouns.remove(word)


전체 형태소 개수: 
240297
전체 명사 개수: 
28568


In [3]:
data = pd.read_csv('natepann.csv')
corpus = [' '.join(i[0].split(' ')[1:]) for i in data.values]

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(corpus):
    print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

[('마라고', 0.6043230432231961), ('하는', 0.5192179878036876), ('시', 0.0), ('어머니', 0.6043230432231961)]
[('대한', 0.5), ('친정과', 0.5), ('시댁의', 0.5), ('온도차', 0.5)]
[('신랑', 0.467258744972966), ('탈모', 0.467258744972966), ('때문에', 0.42876663934768666), ('너무', 0.4014560557786581), ('고민입니다', 0.467258744972966)]
[('애', 0.0), ('맡겼더니', 0.3823028553303722), ('하루', 0.3823028553303722), ('종일', 0.3823028553303722), ('배달', 0.3508092941150033), ('음식', 0.3823028553303722), ('먹인', 0.3823028553303722), ('시누이', 0.3823028553303722)]
[('회사에서', 0.5), ('꾸준히', 0.5), ('오래', 0.5), ('다니는', 0.5), ('법', 0.0)]
[('시험', 0.510175589893032), ('붙더니', 0.510175589893032), ('헤어지자는', 0.510175589893032), ('남자친구', 0.46814805610179616)]
[('남자들', 0.408248290463863), ('생떼', 0.408248290463863), ('쓰는', 0.408248290463863), ('것', 0.0), ('좀', 0.0), ('막아보자', 0.408248290463863), ('청원', 0.408248290463863), ('부탁', 0.408248290463863)]
[('필리핀', 0.5), ('사람이라', 0.5), ('비혼주의가', 0.5), ('됐어요', 0.5)]
[('좋아하는', 0.5172136835355244), ('여자에게는', 0.51721368353

In [4]:
#TF-IDF 
#전체 CSV 파일 직접 입력 시

article_data = pd.read_csv('natepann.csv', encoding='utf-8', header= None)
values = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for value in values:
    as_one = as_one + ' ' + value
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

#단어 빈도 정리
word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

#딕셔너리로 정리
idx2word = {ii: word for ii, word in enumerate(vocab)}

#tf-idf 
tfidf = TfidfVectorizer(max_features = 10, max_df=0.95, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(values)  #size D x V

#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)

['결혼', '남편', '너무', '무조건', '배달', '사람', '싶어', '없는', '있는', '하는']


In [5]:
#TF-IDF 이미 한번 명사 단위로 정리 한 명단으로 처리할 시

as_one = ''
for noun in nouns:
    as_one = as_one + ' ' + noun
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

#단어 빈도 정리
word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

#딕셔너리로 정리
idx2word = {ii: word for ii, word in enumerate(vocab)}

In [6]:
#tf-idf 
tfidf = TfidfVectorizer(max_features = 10, max_df=0.95, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(nouns)  #size D x V

In [7]:
#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)

morphs_dict = {
    '키워드' : tfidf_dict,
}

df2 = pd.DataFrame(morphs_dict)
df2.to_csv('10키워드s.csv', index=False, encoding="utf-8-sig")

['결혼', '남자', '남편', '본인', '부모', '시간', '엄마', '여자', '자기', '친구']


In [8]:
data_array = A_tfidf_sp.toarray()
tfidf_data = pd.DataFrame(data_array, columns=tfidf_dict)
tfidf_data
tfidf_data.shape

(11334, 10)

In [9]:
#토픽 모델링
# LDAModel을 생성합니다.
import tomotopy as tp
model = tp.LDAModel(k=20, alpha=0.1, eta=0.01, min_cf=5)

for i, line in enumerate(nouns): #open('natepann.csv', encoding='utf-8')):
    model.add_doc(line.strip().split()) 

model.train(0) 
print('Total docs:', len(model.docs))
print('Total words:', model.num_words)
print('Vocab size:', model.num_vocabs)

 
for i in range(model.k):
    res = model.get_topic_words(i, top_n=10)
    print('Topic #{}'.format(i), end='\t')
    print(', '.join(w for w, p in res))

Total docs: 11334
Total words: 7514
Vocab size: 527
Topic #0	남편, 결혼, 친구, 엄마, 계속, 아들, 아이, 시간, 남자들, 남자
Topic #1	결혼, 친구, 엄마, 남편, 여자, 부모, 시간, 아이, 남자, 먹이
Topic #2	결혼, 엄마, 남자, 부모, 대하, 친구, 자기, 아들, 고민, 남편
Topic #3	남편, 결혼, 엄마, 친구, 부모, 어도, 자기, 여자, 시간, 사랑
Topic #4	여자, 남자, 아이, 부모, 시간, 결혼, 정말, 시작, 남편, 친구
Topic #5	결혼, 남편, 엄마, 친구, 본인, 남자, 아이, 어도, 사랑, 부모
Topic #6	남편, 결혼, 친구, 여자, 얼굴, 자기, 엄마, 시간, 가요, 아들
Topic #7	남편, 결혼, 친구, 여자, 엄마, 자기, 남자, 시간, 부모, 본인
Topic #8	남편, 결혼, 친구, 남자, 라면, 엄마, 요즘, 아이, 어도, 사랑
Topic #9	남편, 결혼, 친구, 부모, 본인, 사랑, 다시, 애기, 엄마, 시간
Topic #10	결혼, 엄마, 시간, 여자, 부모, 남편, 수준, 정말, 시동생, 친구
Topic #11	결혼, 남편, 아이, 전화, 엄마, 친구, 시간, 여자, 가요, 말씀
Topic #12	결혼, 여자, 남편, 엄마, 남자, 이나, 친구, 부모, 시간, 소리
Topic #13	결혼, 남편, 엄마, 친구, 남자, 여자, 부모, 시간, 자기, 어도
Topic #14	결혼, 남편, 엄마, 여자, 본인, 친구, 서로, 남자, 처음, 연락
Topic #15	남편, 결혼, 엄마, 여자, 본인, 남자, 친구, 배달, 절대, 아빠
Topic #16	남자, 결혼, 엄마, 부모, 여자, 그때, 전화, 시키, 남편, 아이
Topic #17	결혼, 남편, 남자, 엄마, 친구, 여자, 부모, 본인, 시간, 자기
Topic #18	친구, 결혼, 엄마, 여자, 남편, 연락, 남자, 아빠, 유학, 자기
Topic #19	부모, 결혼, 남편, 엄마,